In [ ]:
# %matplotlib notebook

In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

# from scipy.integrate import odeint
################################################
# from autograd.scipy.integrate import odeint
# from autograd.builtins import tuple
################################################

from jax.experimental.ode import odeint
from scipy.optimize import minimize
from scipy.optimize import curve_fit

In [ ]:
import jax 
import jax.numpy as jnp
from jax import jacfwd, jacrev

import numpy as np
# import autograd.numpy as np   # Thinly-wrapped version of Numpy
# from autograd import grad
# from autograd import jacobian

In [ ]:
# from jax.experimental.ode import odeint

# def odeint(func, y0, t, *args, rtol=1.4e-8, atol=1.4e-8, mxstep=jnp.inf, hmax=jnp.inf)

In [ ]:
from multiprocessing import Pool, Process

In [ ]:
# from multiprocessing import multiprocess

In [ ]:
#General plotting functions
#Elimates the left and top lines and ticks in a matplotlib plot 
def PlotStyle(Axes,Title):
    
    Axes.spines['top'].set_visible(False)
    Axes.spines['right'].set_visible(False)
    Axes.spines['bottom'].set_visible(True)
    Axes.spines['left'].set_visible(True)
    Axes.xaxis.set_tick_params(labelsize=14)
    Axes.yaxis.set_tick_params(labelsize=14)
    Axes.set_title(Title)

#Lollipop plot based on the python graph gallery implementation 
def LollipopPlot(Fig,Time,Data,Regression):
    
    cTime=Time
    cData=Data
    cRegression=Regression
    
    ax=Fig.gca()

    (markers, stemlines, baseline) = ax.stem(cTime, cData,bottom=-0.4,label='Data',basefmt=" ")
    plt.setp(stemlines, linestyle="-", color="red", linewidth=0.5,alpha=0.5 )
    plt.setp(markers, color="red",alpha=0.75 )

    ax.plot(cTime,cRegression,'b-',label='Regression',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])
  
    ax.set_ylabel('Normalized Population',fontsize=16,fontweight='bold')
    ax.set_xlabel('Time',fontsize=16,fontweight='bold')
    ax.legend(loc=0,fontsize=14)
    ax.set_ylim(-0.4,3.4)
    PlotStyle(ax,'')

## Parameter Estimation of Non-Linear ODE system

In [ ]:
#Let's consider the Lodka-Volterra model, also known as the predator-prey model. (https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations)

<!-- $\frac{\partial R}{\partial t} = R(\alpha - \beta W),  \hspace{1cm} W(0)=W_{0} \\$
<!-- $\frac{\partial W}{\partial t} = -W(\gamma - \delta R),\hspace{1cm} R(0)=R_{0}$ --> -->




In [ ]:
def RMS_error(y1,y2):
    term1 = np.sum((y1-y2)**2)
    return np.sqrt(term1)

In [ ]:
#Non linear model
def MakeModel(k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1, StateVariables):
    Variables = StateVariables
    Equation1= ((k_1*(Variables[1]/(k_n1+Variables[1]))) * (J_11/(J_11+Variables[0]))) - (phi_pRB*Variables[0])
    Equation2= k_p + ((k_2*((a**2 + Variables[1]**2) / (k_n2**2 + Variables[1]**2))) * (J_12/(J_12+Variables[0]))) \
               - phi_E2F1*Variables[1] 
    
    return jnp.array([Equation1,Equation2])

#Integration time
GlobalSolverTime=jnp.linspace(0,800,num=20)

#Parameters of the model 
k_1 = 1
k_n1=0.5
J_11=0.5
phi_pRB=0.005
k_p = 0.05
k_2 = 1.6
a = 0.04
k_n2 = 4
J_12 = 5
phi_E2F1 = 0.1

# true_parameter_array = jnp.array([k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1])
true_parameter_array = np.array([k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1])
# ESTIMATE: k_1, J_11, k_n2, J_12

num_parameters = 10 #Number of parameters in total
est_par_index = [0,2,7,8]
num_unknown_parameters = len(est_par_index)
#True initial conditions 
Int=jnp.array([1.0,5.0])  # [pRB(0), E2F1(0)] 

######################################################
# #Model Solution 
# def ODEModel(StateVariables,t):

#     return MakeModel(k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1,StateVariables)

# Solution=odeint(ODEModel,Int,SolverTime)
########################################################

def ODEModel(StateVariables,t,K):
    k_1,k_n1,J_11,phi_pRB,k_p,k_2,a,k_n2,J_12,phi_E2F1 = K

    return MakeModel(k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1,StateVariables)

Solution = odeint(ODEModel,Int,GlobalSolverTime,tuple([k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1]))

In [ ]:
#Visualization
plt.figure(1,figsize=(9,6))

plt.plot(GlobalSolverTime,Solution[:,0],'b-',label='pRB',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])
plt.plot(GlobalSolverTime,Solution[:,1],'g-',label='E2F1',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])

plt.xlabel('Time',fontsize=16,fontweight='bold')
plt.ylabel('Concentration',fontsize=16,fontweight='bold')
plt.legend(loc=0,fontsize=14)

ax=plt.gca()
PlotStyle(ax,'')


In [ ]:
# concat_time = np.concatenate((SolverTime,SolverTime))

In [ ]:
###############################################################################
#                            Data generation  
###############################################################################
def MakeNoisyData(Data,Noise):
    
    return [val+cal for val,cal in zip(Data,Noise)]

GaussianNoise1=[np.random.normal(0,0.001) for val in Solution[:,0]]  
GaussianNoise2=[np.random.normal(0,0.001) for val in Solution[:,1]]  
Signal1=MakeNoisyData(Solution[:,0],GaussianNoise1)
Signal2=MakeNoisyData(Solution[:,1],GaussianNoise2)

Signal1 = jnp.asarray(Signal1)
Signal2 = jnp.asarray(Signal2)

concat_noisy_signal = jnp.concatenate((Signal1,Signal2))

In [ ]:
num_concatdata_points = concat_noisy_signal.shape[0]
num_concatdata_points

In [ ]:
jnp.asarray(Signal1).shape

In [ ]:
# ESTIMATE: k_1, J_11, k_n2, J_12
###############################################################################
#                    ODE system model fitting  
###############################################################################
# LATER: Consider the initial parameters as two extra parameters (so 10+2 = 12 parameters in total)
#ModelSolver is the forward solver/integrator
def ModelSolver(t,k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1,InitialConditions):
    
    ck_1 =  k_1
    ck_n1 =  k_n1
    cJ_11 = J_11
    cphi_pRB =  phi_pRB
    ck_p = k_p
    ck_2 = k_2
    ca = a
    ck_n2 = k_n2 
    cJ_12 = J_12
    cphi_E2F1 = phi_E2F1
    
    cInit=InitialConditions
    
    def LocalModel(cInit,t):
        
        return MakeModel(ck_1,ck_n1,cJ_11,cphi_pRB,ck_p,ck_2,ca,ck_n2,cJ_12,cphi_E2F1,cInit)
    
    Solution=odeint(LocalModel,cInit,t)
    
    return jnp.concatenate((Solution[:,0],Solution[:,1]))

def ModelFit(t,k_1,J_11,k_n2,J_12): #change these if the parameters to be estimated change
    #the values of the other parameters are obtained from their global scope 
    t1,t2 =  t #t1 and t2 should be identical
    return ModelSolver(t1,k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1,Int)

# p0 = np.random.rand(4)
# p0 = np.array([0.45467901, 0.99152412, 0.89004968, 0.48321337])
p0 = np.array([0.91724398, 0.34276907, 0.14016909, 0.85728149])
weights = np.ones(num_concatdata_points) #Initially, all weights are 1
sigma_now = 1/np.sqrt(weights)
ModelParams01=curve_fit(ModelFit,(GlobalSolverTime,GlobalSolverTime),concat_noisy_signal,p0,sigma=sigma_now)

# #Parameters of the model 
# k_1 = 1
# k_n1=0.5
# J_11=0.5
# phi_pRB=0.005
# k_p = 0.05
# k_2 = 1.6
# a = 0.04
# k_n2 = 4
# J_12 = 5
# phi_E2F1 = 0.1


# FitSolution=ModelSolver(SolverTime,ModelParams01[0][0],ModelParams01[0][1],ModelParams01[0][2],ModelParams01[0][3],Int) #fitted solution

In [ ]:
# p0 = array([0.45467901, 0.99152412, 0.89004968, 0.48321337]) doesn't throw warning.

In [ ]:
ModelParams01

In [ ]:
delta1 = np.array([ModelParams01[0][0]-k_1, ModelParams01[0][1]-J_11,ModelParams01[0][2]-k_n2,ModelParams01[0][3]-J_12])

In [ ]:
np.sqrt(delta1**2) #root squared distance from true values

In [ ]:
100*delta1/ModelParams01[0]  #BIAS, in percentages

In [ ]:
# ESTIMATE: k_1, J_11, k_n2, J_12
# p0

In [ ]:
ModelParams01[0]

In [ ]:
fitSolverTime = np.linspace(0,800,num=20)

#Visualize fit solution
k_1_fit = ModelParams01[0][0]
J_11_fit = ModelParams01[0][1]
k_n2_fit = ModelParams01[0][2]
J_12_fit = ModelParams01[0][3]
Fit = ModelSolver(fitSolverTime,k_1_fit,k_n1,J_11_fit,phi_pRB,k_p,k_2,a ,k_n2_fit,J_12_fit,phi_E2F1,Int)

prb_fit = Fit[:(int(len(Fit)/2))]
e2f1_fit= Fit[int((len(Fit)/2)):]

In [ ]:
prb_fit.shape

In [ ]:
#Visualization
plt.figure(1,figsize=(9,6))
plt.plot(fitSolverTime,prb_fit,'b-',label='pRB',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])
plt.plot(fitSolverTime,e2f1_fit,'g-',label='E2F1',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])

plt.xlabel('Time',fontsize=16,fontweight='bold')
plt.ylabel('Concentration',fontsize=16,fontweight='bold')
plt.legend(loc=0,fontsize=14)


ax=plt.gca()
PlotStyle(ax,'')


In [ ]:
RMS_error(prb_fit,Solution[:,0]) #RMS error in pRB concentration

In [ ]:
RMS_error(e2f1_fit,Solution[:,1]) #RMS error in E2F1 concentration

In [ ]:
100*RMS_error(prb_fit,Solution[:,0])/Solution[:,0] #Percentage error in pRB concentrations at time points

In [ ]:
100*RMS_error(e2f1_fit,Solution[:,1])/Solution[:,1] #Percentage error in E2F1 concentrations at time points

In [ ]:
#ModelSolver is the forward solver/integrator
#True initial conditions 
InitialConditions=np.array([1.0,5.0])  # [pRB(0), E2F1(0)] 
#Redefine SolverTime
# SolverTime=np.linspace(0,800,num=10)
def ModelSolver_g(K):
    k_1,k_n1,J_11,phi_pRB,k_p,k_2,a ,k_n2,J_12,phi_E2F1 = K
    t = GlobalSolverTime #defined globally
    ck_1 =  k_1
    ck_n1 =  k_n1
    cJ_11 = J_11
    cphi_pRB =  phi_pRB
    ck_p = k_p
    ck_2 = k_2
    ca = a
    ck_n2 = k_n2 
    cJ_12 = J_12
    cphi_E2F1 = phi_E2F1
    
    cInit=InitialConditions
    
    def LocalModel(cInit,t,K):
        ck_1,ck_n1,cJ_11,cphi_pRB,ck_p,ck_2,ca,ck_n2,cJ_12,cphi_E2F1 = K
        
        return MakeModel(ck_1,ck_n1,cJ_11,cphi_pRB,ck_p,ck_2,ca,ck_n2,cJ_12,cphi_E2F1,cInit)
    
    Solution=odeint(LocalModel,cInit,t,tuple((ck_1,ck_n1,cJ_11,cphi_pRB,ck_p,ck_2,ca,ck_n2,cJ_12,cphi_E2F1)))
    
    return jnp.concatenate((Solution[:,0],Solution[:,1]))

In [ ]:
# jacobian_func = jacobian(ModelSolver_g,(0)) #Define the jacobian outputting function, AUTOGRAD function
jacobian_fn = jacrev(ModelSolver_g) 
# jacobian_fn = jacfwd(ModelSolver_g) 

In [ ]:
# output_array = jnp.asarray(ModelSolver_g(jnp.array([1.0,0.5,0.5,0.005,0.05,1.6,0.04,4.0,5.0,0.1])))

In [ ]:
# output_array.shape

In [ ]:
# #Parameters of the model 
# k_1 = 1
# k_n1=0.5
# J_11=0.5
# phi_pRB=0.005
# k_p = 0.05
# k_2 = 1.6
# a = 0.04
# k_n2 = 4
# J_12 = 5
# phi_E2F1 = 0.1

#Let's now take the gradient of ModelSolver with respect to the unknown parameters k_1, J_11, k_n2, J_12

In [ ]:
# jacobian_output = jacobian_func(np.array([1.0,0.5,0.5,0.005,0.05,1.6,0.04,4.0,5.0,0.1]))
jacobian_output = jacobian_fn(jnp.array([1.0,0.5,0.5,0.005,0.05,1.6,0.04,4.0,5.0,0.1]))

In [ ]:
jacobian_output = np.asarray(jacobian_output)

In [ ]:
type(jacobian_output)

In [ ]:
jacobian_output.shape  #[num_data_points,num_parameters]

In [ ]:
# ESTIMATE: k_1, J_11, k_n2, J_12
# est_par_index = [0,2,7,8]

In [ ]:
#concatenated data is assumed to have Gaussian noise added
def ParameterEstimation(current_weights,S2_concatdata,S2_SolverTime):
    
    # p0 = np.random.rand(4)
    p0 = np.array([0.45467901, 0.99152412, 0.89004968, 0.48321337])
#     weights = np.ones(S2_concatdata) #Initially, all weights are 1
    zero_idx = np.argwhere(current_weights < 1e-16)
    current_weights[zero_idx] = 1e-14
    current_sigma = 1/np.sqrt(current_weights)
#     print(current_sigma.shape)
#     print(S2_concatdata.shape)
    OptimPar_and_covMatrix=curve_fit(ModelFit,(S2_SolverTime,S2_SolverTime),S2_concatdata,p0,sigma=current_sigma)
    
    return OptimPar_and_covMatrix #a tuple, parameters estimated from S2 data points

In [ ]:
def CalculateJacobian(ParamEstOutput):
    parameter_array = true_parameter_array #true_parameter_array is defined globally
    parameter_array[est_par_index] = ParamEstOutput[0] #est_par_index is defined globally
    jacobian_output = jacobian_func(parameter_array)
    
    JacobianMatrix = jacobian_output[:,est_par_index]  #this is our J
    
    return JacobianMatrix #numpy array of shape [num_concat_data_points, num_estimated_parameters]

In [ ]:
#input index of concatenated data point that we want the weight of
def ComputeWeights(iw):
    # "i" is the index of data points S1
    #JTJ = np.matmul(J.T,J)
    #I = JTJ #approximate total Fisher Information Matrix at optimal parameters. 
    
    #for i in range(num_concatdata_points):
    i = int(iw[0])
    current_weights = iw[1:]
#     print(current_weights.shape)
#     print(i)
    S2_data_signal1 = np.delete(Signal1,i)
    S2_data_signal2 = np.delete(Signal2,i)
    S2_SolverTime = np.delete(GlobalSolverTime,i)
    S2_weights = np.delete(current_weights, i)
    
    S2_concat_data = np.concatenate((S2_data_signal1,S2_data_signal2))
    S2_weights = np.concatenate((S2_weights,S2_weights))
#     print(S2_concat_data.shape)
#     print(S2_weights.shape)
    #returns a tuple
    ParamEstimates_andCovMatrix = ParameterEstimation(S2_weights,S2_concat_data,S2_SolverTime) 
    
    #Now to compute the Jacobian using the parameters estimated from S2
    
    unknown_parameter_estimates = ParamEstimates_andCovMatrix[0]
    temp_parameter_array = np.asarray(true_parameter_array)
    temp_parameter_array[est_par_index] = unknown_parameter_estimates
    
    #this computes Jacobian of ALL points with respect to ALL parameters
    
    jacobian_output = jacobian_fn(temp_parameter_array) 
    jacobian_output = np.asarray(jacobian_output)
    J = jacobian_output[:,est_par_index]  #this is our J, est_par_index is defined globally
    #Now to form J^T * W * J
    W = np.diag(current_weights)
    ##############################################################################
#     WJ = np.matmul(W,J) #the "WJ" part in J^T * W * J
#     #this is our J_T*W*J which is the Fisher information matrix approximation
#     #This is also I, the total Fisher Information Matrix approximation, using the estimated pars from S2
#     JTWJ = np.matmul(J.T,WJ)   #I
#     ###############################################################################
    
    #First form I_S1
    S1_row_in_jacobian = J[i,:].reshape(1,num_unknown_parameters) #num_unknown_parameters defined globally
    #W should be diagonal, JTWJ_S1 should have shape [num_unknown_parameters,num_unknown_parameters]
    JTWJ_S1 = W[i,i] * np.matmul(S1_row_in_jacobian.T,S1_row_in_jacobian) 
    I_S1 = JTWJ_S1  
    
    #Now form I_S2
    num_time_points = np.arange(len(Signal1))
    S2_indices = (np.setdiff1d(num_time_points,i)).astype(int)
#     print(type(S2_indices))
    #If num of S2 points was 1, you'd need to reshape this. But it probably isn't 1, UNLESS you have two time data points
    S2_rows_in_jacobian = J[S2_indices,:] 
    W_forS2 = np.diag(current_weights[S2_indices])
    WJ_S2 = np.matmul(W_forS2,S2_rows_in_jacobian)
    I_S2 = np.matmul(S2_rows_in_jacobian.T,WJ_S2)
    
    
#     I_S1_I_S2 = np.matmul(I_S1,I_S2)
    
#     identity_matrix = np.identity(I_S1_I_S2.shape[0])
#     I_S1_I_S2_inv = np.linalg.solve(I_S1_I_S2,identity_matrix)
    identity_matrix = np.identity(I_S2.shape[0])
    I_S2_inv = np.linalg.solve(I_S2,identity_matrix)
    
#     uncertainty_of_S1_given_S2 = (1.0/num_unknown_parameters) * np.trace(I_S1_I_S2_inv)
    uncertainty_of_S1_given_S2 = (1.0/num_unknown_parameters) * np.trace(np.matmul(I_S1,I_S2_inv))
    
    new_S1_weight = uncertainty_of_S1_given_S2
    
    return new_S1_weight

In [ ]:
current_weights = np.ones([len(Signal1),len(Signal1)])

In [ ]:
current_weights.shape

In [ ]:
i_array = np.arange(len(Signal1)).reshape(len(Signal1),1)

In [ ]:
i_array

In [ ]:
arr = np.hstack((i_array,current_weights))

In [ ]:
arr.shape

In [ ]:
arr[4,:]

In [ ]:
arr = arr[:3,:

In [ ]:
arr.shape

In [ ]:
ComputeWeights(arr[2, :])

In [ ]:
len(Signal1)

In [ ]:
# w_array = np.zeros(20)
# for index,item in enumerate(arr):
# #     print(index)
#     w_array[index] = ComputeWeights(item)

In [ ]:
import multiprocess as mp

In [ ]:
import time

In [ ]:
# %%timeit 
# ComputeWeights(arr[2, :])  #35 seconds per point in each iteration


In [ ]:
(35*20)/60

In [ ]:
# start = time.time()
# # ##### START SIMULATION ####
# # if __name__ == "__main__":
# #     p = Pool()   
# #     result1 = p.map(ComputeWeights,arr)
# #     p.close()
# #     p.join()

# # result1 = np.array(result1)
# # result1 = result1.squeeze()


# with mp.Pool(8) as pool:
#     result1 = (pool.map(ComputeWeights, arr))
    
# end = time.time()
# print(end-start)

In [ ]:
# result1 = np.array(result1)
# result1 = result1.squeeze()

In [ ]:
# result1

In [ ]:
# result1.shape

In [ ]:
# xtemp = np.ones([1,2])
# np.repeat(xtemp,2,axis=0)

In [ ]:
# np.repeat(result1.reshape(1,3),3,axis=0)

In [ ]:
# len(result1)

In [ ]:
arr.shape

In [ ]:
################################ Iterative process ###################################
iterations = 1

weights_array = np.ones([iterations,len(Signal1)])
arr_iteration = arr
start = time.time()
for i in np.arange(iterations):
    with mp.Pool(8) as pool:
        result1 = (pool.map(ComputeWeights, arr_iteration))
    
    result1 = np.array(result1)
    result1 = result1.squeeze()
    
    weights_array[i,:] = result1
    
    arr_iteration = np.repeat(result1.reshape(1,len(result1)),len(Signal1),axis=0) #Signal1 and result1 should have the same length
    
    arr_iteration = np.hstack((i_array,arr_iteration))

    
end = time.time() 

print(end-start)



In [ ]:
35*20

In [ ]:
700/230

In [ ]:
arr.shape

In [ ]:
start = time.time()
result_test = ComputeWeights, arr
end = time.time()

In [ ]:
file = np.load("FirstRun.npy")

In [ ]:
file.shape

In [ ]:
file[3,:]

In [ ]:
arr_file = np.repeat(file[3,:].reshape(1,len(file[3,:])),len(Signal1),axis=0)

In [ ]:
arr_file.shape

In [ ]:
arr_file = np.hstack((i_array,arr_file))

In [ ]:
arr_file.shape

In [ ]:
arr_file[3,:]

In [ ]:
################################ Iterative process (Start from file)###################################
iterations = 3

weights_array = np.ones([iterations,len(Signal1)])
arr_iteration = arr_file
start = time.time()
for i in np.arange(iterations):
    with mp.Pool(8) as pool:
        result1 = (pool.map(ComputeWeights, arr_iteration))
    
    result1 = np.array(result1)
    result1 = result1.squeeze()
    
    weights_array[i,:] = result1
    
    arr_iteration = np.repeat(result1.reshape(1,len(result1)),len(Signal1),axis=0) #Signal1 and result1 should have the same length
    
    arr_iteration = np.hstack((i_array,arr_iteration))

    
end = time.time() 

print(end-start)

In [ ]:
weights_array.shape

In [ ]:
np.save("SecondRun.npy",weights_array)

In [ ]:
plt.figure(1,figsize=(9,6))
plt.scatter(GlobalSolverTime,file[0,:],label="weights")

plt.yscale('log')
plt.legend(loc=0,fontsize=14)
plt.xlabel('Time [mins]',fontsize=16,fontweight='bold')
plt.ylabel('Weights',fontsize=16,fontweight='bold')

In [ ]:
plt.figure(1,figsize=(9,6))
plt.scatter(GlobalSolverTime,file[1,:],label='weights')

plt.yscale('log')
plt.legend(loc=0,fontsize=14)
plt.xlabel('Time [mins]',fontsize=16,fontweight='bold')
plt.ylabel('Weights',fontsize=16,fontweight='bold')


In [ ]:
plt.figure(1,figsize=(9,6))
plt.scatter(GlobalSolverTime,file[2,:], label='weights')

plt.yscale('log')
plt.legend(loc=0,fontsize=14)
plt.xlabel('Time [mins]',fontsize=16,fontweight='bold')
plt.ylabel('Weights',fontsize=16,fontweight='bold')

In [ ]:
plt.figure(1,figsize=(9,6))
plt.scatter(GlobalSolverTime,file[3,:],label='weights')

plt.yscale('log')
plt.legend(loc=0,fontsize=14)
plt.xlabel('Time [mins]',fontsize=16,fontweight='bold')
plt.ylabel('Weights',fontsize=16,fontweight='bold')

In [ ]:
plt.figure(1,figsize=(9,6))
plt.scatter(np.delete(GlobalSolverTime,1),np.delete(file[3,:],1),label="weights")
plt.title("Non log plot")
# plt.yscale('log')
plt.legend(loc=0,fontsize=14)
plt.xlabel('Time [mins]',fontsize=16,fontweight='bold')
plt.ylabel('Weights',fontsize=16,fontweight='bold')

In [ ]:
# collect.shape

In [ ]:
collect = np.zeros([4,20])
for item in np.arange(4): 
    collect[item,:] = file[item,:]
        
# plt.scatter(GlobalSolverTime,file[item,:],label='weights')
    

In [ ]:
plt.figure(1,figsize=(9,6))
plt.scatter(np.arange(4),collect[:,0],label='iteration 1')
plt.scatter(np.arange(4),collect[:,1],label='iteration 2')
plt.scatter(np.arange(4),collect[:,2],label='iteration 3')
plt.scatter(np.arange(4),collect[:,3],label='iteration 4')
plt.yscale('log')
plt.axhline(y = 0, color = 'r', linestyle = '-')
plt.legend()

In [ ]:
# fig0,axes = plt.subplots(1,2,figsize=(16,8))

# plt.scatter(GlobalSolverTime,file[3,:])
plt.figure(1,figsize=(9,6))
plt.plot(GlobalSolverTime,file[3,:],'-o',label='Avg. weights')
plt.yscale('log')
plt.legend(loc=0,fontsize=14)
plt.xlabel('Time [mins]',fontsize=16,fontweight='bold')
plt.ylabel('Weights',fontsize=16,fontweight='bold')
plt.show()

In [ ]:
Solution[:,0].shape == GlobalSolverTime.shape

In [ ]:
GlobalSolverTime.shape

In [ ]:
file[3,:].shape

In [ ]:
# fig, axs = plt.subplots(2)
# fig.suptitle('Vertically stacked subplots')
# axs[0].plot(x, y)
# axs[1].plot(x, -y)


fig0,axes = plt.subplots(1,2,figsize=(16,8))

# plt.scatter(GlobalSolverTime,file[3,:])
# plt.figure(1,figsize=(9,6))
axes[0].plot(GlobalSolverTime,file[3,:],'-o',label='Avg. weights')
axes[0].set_yscale('log')
axes[0].legend(loc=0,fontsize=14)
axes[0].set_xlabel('Time [mins]',fontsize=16,fontweight='bold')
axes[0].set_ylabel('Weights',fontsize=16,fontweight='bold')


axes[1].plot(GlobalSolverTime,Solution[:,0],'-o',label='pRB')
# axes[1].set_yscale('log')
axes[1].legend(loc=0,fontsize=14)
axes[1].set_xlabel('Time [mins]',fontsize=16,fontweight='bold')
axes[1].set_ylabel('Weights',fontsize=16,fontweight='bold')

axes[1].plot(GlobalSolverTime,Solution[:,1],'-o',label='E2F1')
# axes[1].set_yscale('log')
axes[1].legend(loc=0,fontsize=14)
axes[1].set_xlabel('Time [mins]',fontsize=16,fontweight='bold')
axes[1].set_ylabel('Concentration',fontsize=16,fontweight='bold')

plt.show()



In [ ]:
file[3,:]

In [ ]:
ind = np.argpartition(file[3,:], -4)[-4:]

In [ ]:
ind

### Curve Fitting with selected time points

In [ ]:
# selected_idx = np.argwhere(file[3,:] > 2.7e-7)
selected_idx = np.argpartition(file[3,:], -7)[-7:]

In [ ]:
selected_idx.shape

In [ ]:
Time = GlobalSolverTime[selected_idx].flatten()

In [ ]:
#Visualization
plt.figure(1,figsize=(9,6))

plt.plot(GlobalSolverTime,Solution[:,0],'b-',label='pRB',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])
plt.plot(GlobalSolverTime,Solution[:,1],'g-',label='E2F1',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])

plt.xlabel('Time',fontsize=16,fontweight='bold')
plt.ylabel('Concentration',fontsize=16,fontweight='bold')
plt.legend(loc=0,fontsize=14)

xcoords = Time
for xc in Time:
    plt.axvline(x=xc)

ax=plt.gca()
PlotStyle(ax,'')


In [ ]:
###############################################################################
#                            Data generation  
###############################################################################
new_CleanSignal1 = Solution[:,0][selected_idx]
new_CleanSignal2 = Solution[:,1][selected_idx]
def MakeNoisyData(Data,Noise):
    
    return [val+cal for val,cal in zip(Data,Noise)]

GaussianNoise1=[np.random.normal(0,0.001) for val in new_CleanSignal1]  
GaussianNoise2=[np.random.normal(0,0.001) for val in new_CleanSignal2]  
Signal1=MakeNoisyData(new_CleanSignal1,GaussianNoise1)
Signal2=MakeNoisyData(new_CleanSignal2,GaussianNoise2)

new_Signal1 = jnp.asarray(new_CleanSignal1).flatten()
new_Signal2 = jnp.asarray(new_CleanSignal2).flatten()

new_concat_noisy_signal = jnp.concatenate((new_Signal1,new_Signal2))



# def MakeNoisyData(Data,Noise):
    
#     return [val+cal for val,cal in zip(Data,Noise)]

# GaussianNoise1=[np.random.normal(0,0.001) for val in Solution[:,0]]  
# GaussianNoise2=[np.random.normal(0,0.001) for val in Solution[:,1]]  
# Signal1=MakeNoisyData(Solution[:,0][selected_idx],GaussianNoise1)
# Signal2=MakeNoisyData(Solution[:,1][selected_idx],GaussianNoise2)

# Signal1 = jnp.asarray(Signal1)
# Signal2 = jnp.asarray(Signal2)

# concat_noisy_signal = jnp.concatenate((Signal1,Signal2))


In [ ]:
Time.shape

In [ ]:
new_concat_noisy_signal.flatten().shape

In [ ]:
new_num_concat_data_points = len(new_concat_noisy_signal)

In [ ]:
Time.shape

In [ ]:
# new_sigma_now.shape

In [ ]:
new_num_concat_data_points

In [ ]:
# p0 = np.random.rand(4)
# p0 = np.array([0.45467901, 0.99152412, 0.89004968, 0.48321337])
p0 = jnp.array([0.91724398, 0.34276907, 0.14016909, 0.85728149])
new_weights =np.ones(new_num_concat_data_points) #Initially, all weights are 1
# new_sigma_now = 1/jnp.sqrt(new_weights)


In [ ]:
#NOW ASSUME ALL WEIGHTS ARE 1 AGAIN - ASSUME ALL EQUALLY IMPORTANT BUT FIT USING MOST IMPORTANT POINTS

ModelParams02=curve_fit(ModelFit,(Time,Time),new_concat_noisy_signal.flatten(),p0,sigma=new_weights)
# ModelParams01=curve_fit(ModelFit,(GlobalSolverTime[selected_idx],GlobalSolverTime[selected_idx]),concat_noisy_signal.flatten(),p0,sigma=sigma_now)

In [ ]:
# #Parameters of the model  # ESTIMATE: k_1, J_11, k_n2, J_12
#******************* k_1 = 1
# k_n1=0.5
#****************** J_11=0.5
# phi_pRB=0.005
# k_p = 0.05
# k_2 = 1.6
# a = 0.04
#****************** k_n2 = 4
#****************** J_12 = 5
# phi_E2F1 = 0.1

ModelParams02[0] #new estimates

In [ ]:
# ModelParams01[0] #PREVIOUS estimate with ALL time points

In [ ]:
ModelParams02[1] #COVARIANCE matrix of NEW estimates at selected time points

In [ ]:
# #Parameters of the model  # ESTIMATE: k_1, J_11, k_n2, J_12
# k_1 = 1
# k_n1=0.5
# J_11=0.5
# phi_pRB=0.005
# k_p = 0.05
# k_2 = 1.6
# a = 0.04
# k_n2 = 4
# J_12 = 5
# phi_E2F1 = 0.1

delta = np.array([ModelParams02[0][0]-k_1, ModelParams02[0][1]-J_11,ModelParams02[0][2]-k_n2,ModelParams02[0][3]-J_12])


In [ ]:
100*delta/ModelParams02[0] #BIAS, in percentages

In [ ]:
fitSolverTime2 = np.linspace(0,800,num=20)

#Visualize fit solution
k_1_fit2 = ModelParams02[0][0]
J_11_fit2 = ModelParams02[0][1]
k_n2_fit2 = ModelParams02[0][2]
J_12_fit2 = ModelParams02[0][3]
Fit2 = ModelSolver(fitSolverTime2,k_1_fit2,k_n1,J_11_fit2,phi_pRB,k_p,k_2,a ,k_n2_fit2,J_12_fit2,phi_E2F1,Int)

prb_fit2 = Fit2[:(int(len(Fit2)/2))]
e2f1_fit2= Fit2[int((len(Fit2)/2)):]

In [ ]:
#Visualization
plt.figure(1,figsize=(9,6))
plt.plot(fitSolverTime2,prb_fit2,'b-',label='pRB',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])
plt.plot(fitSolverTime2,e2f1_fit2,'g-',label='E2F1',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])

plt.xlabel('Time',fontsize=16,fontweight='bold')
plt.ylabel('Concentration',fontsize=16,fontweight='bold')
plt.legend(loc=0,fontsize=14)

#########

#########
ax=plt.gca()
PlotStyle(ax,'')


In [ ]:
plt.figure(1,figsize=(9,6))
plt.plot(fitSolverTime2,Solution[:,0],'b-',label='pRB',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])
plt.plot(fitSolverTime2,Solution[:,1],'g-',label='E2F1',path_effects=[path_effects.SimpleLineShadow(alpha=0.2,rho=0.2),
                       path_effects.Normal()])

plt.xlabel('Time',fontsize=16,fontweight='bold')
plt.ylabel('Concentration',fontsize=16,fontweight='bold')
plt.title("Data")
plt.legend(loc=0,fontsize=14)

In [ ]:
RMS_error(prb_fit2,Solution[:,0]) #RMS error in pRB concentration

In [ ]:
RMS_error(e2f1_fit2,Solution[:,1])  #RMS error in E2F1 concentration

In [ ]:
# 100*RMS_error(prb_fit2,Solution[:,0])/Solution[:,0] #Percentage error in pRB concentrations at time points

In [ ]:
# 100*RMS_error(e2f1_fit2,Solution[:,0])/Solution[:,0] #Percentage error in pRB concentrations at time points

# G1/S transition with 6 unknown parameters

In [ ]:
###############################################################################
#                    Standard parameter estimation  
###############################################################################

fig=plt.figure(2,figsize=(9,6))

LollipopPlot(fig,SolverTime,WhiteSignal,FitSolution)

In [ ]:
#The Levenberg-Marquardt algorithm is used by curve_fit to minimize the squared error. Dimensionality of objective
# function is 4 while that of the model is 2, and the default curve_fit initial guesses (all 1's) fail to find an adequate solution

In [ ]:
#Let's do a random search of the initial guess, and make use of the minimize function from scipy.optimize

##############################################################################
#                    Performance measurement
###############################################################################

#Measures the squared error between the data and the fit given an initial parameter guess
def SquaredError(InitialParameterGuess):
    
    try:
        #Some guesses will result in an error or an invalid integration 
        #this ensures that the error is estimated only for valid solutions 
        cModelParams=curve_fit(ModelFit,SolverTime,WhiteSignal,p0=InitialParameterGuess)
        cSolution=ModelSolver(SolverTime,cModelParams[0][0],cModelParams[0][1],cModelParams[0][2],cModelParams[0][3],Int)
    
        error=[(val-sal)**2 for val,sal in zip(cSolution,WhiteSignal)]
        
    except RuntimeError:
        
        #Based on the scale of the data the following list of values will be large enough to be rejected as a solution 
        error=[10,10]
    
    return sum(error)

In [ ]:
'''
Now let's construct a random search algorithm, which will generate a list with "n" different random initial guesses and select the best approximation
'''
###############################################################################
#                    Random Search of initial values  
###############################################################################

#Random selection of the initial guess values 
def RandomSearch(Iterations):
    
    nIt=Iterations 
    GuessContainer=[]
    ErrorContainer=[]
    
    for k in range(nIt):
        
        lGuess=[np.random.uniform(low=0,high=1) for val in range(4)]  #we have 4 parameters to estimate hence range(4), parameters are between 0 to 1
        lerror=SquaredError(lGuess)
        
        GuessContainer.append(lGuess) #Contains the list of random initial values 
        ErrorContainer.append(lerror) #List of the errors 
    
    minError=np.min(ErrorContainer) #Min error value 
    minLocation=[j for j in range(nIt) if ErrorContainer[j]==minError] #Returns the location of the min value in the list
    
    bestGuess=GuessContainer[minLocation[0]] #Best initial value guess 
    
    return ErrorContainer,bestGuess
    
    
errs,bG=RandomSearch(10)     #let's search 10 times randomly

In [ ]:
###############################################################################
#                   Initial values Random Search Fitting 
###############################################################################

ModelParams02=curve_fit(ModelFit,SolverTime,WhiteSignal,p0=bG)
FitSolution2=ModelSolver(SolverTime,ModelParams02[0][0],ModelParams02[0][1],ModelParams02[0][2],ModelParams02[0][3],Int)

In [ ]:
'''
Parameters of the model 
alpha=0.25
beta=0.55
gamma=0.3
delta=0.6
'''
ModelParams02[0][0]

In [ ]:
ModelParams02[0][1]

In [ ]:
ModelParams02[0][2]

In [ ]:
ModelParams02[0][3]

In [ ]:
#We can see that the estimated parameters are close to the input parameters
#Let's see their covariance matrix:
ModelParams02[1]

In [ ]:
sd_array = np.sqrt(np.diag(ModelParams02[1])) #standard deviations of estimated parameters
sd_array

In [ ]:
np.divide(100*sd_array,ModelParams02[0]) #Standard deviations as percentage of estimated parameter values, shows good fit

In [ ]:
###############################################################################
#                              Visualization  
###############################################################################

fig=plt.figure(3,figsize=(9,6))

LollipopPlot(fig,SolverTime,WhiteSignal,FitSolution2)

SUCCESS, now let's try a second approach:

1. Minimize the squared error using an algorithm and get estimates for the parameters.
2. Use the result found in (1) as the initial guess for another least squares optimization (use it in curve_fit)

In [ ]:
'''
A good algorithm for non-differentiable objective functions, or challenging non-linear and noisy optimization problems
is the Nelder-Mead optimization algorithm. It is often used for multidimensional nonlinear function optimization problems, although it can get stuck in local optima.
My reference for this was: https://machinelearningmastery.com/how-to-use-nelder-mead-optimization-in-python/
'''

###############################################################################
#                    Initial values Nelder-Mean Fitting
###############################################################################

#Vector of random initial values 
g0=[np.random.uniform(low=0,high=1) for val in range(4)] 

#Minimization of the squared error by Nelder-Mead 
res = minimize(SquaredError, g0, method='nelder-mead',options={'xtol': 1e-3, 'maxiter':100,'disp': False})



In [ ]:
res.success

In [ ]:
res.message

In [ ]:
'''
Parameters of the model 
alpha=0.25
beta=0.55
gamma=0.3
delta=0.6
'''

res.x    #the 4 parameter estimates

In [ ]:
#Now use the result obtained as the initial guess for curve_fit

ModelParams03=curve_fit(ModelFit,SolverTime,WhiteSignal,p0=res.x)

FitSolution3=ModelSolver(SolverTime,ModelParams03[0][0],ModelParams03[0][1],ModelParams03[0][2],ModelParams03[0][3],Int) #the fitted curve

In [ ]:
ModelParams03[0] #Much better estimates

In [ ]:
ModelParams03[1] #Covariance matrix

In [ ]:
sd_array = np.sqrt(np.diag(ModelParams03[1])) #standard deviations of estimated parameters
sd_array

In [ ]:
np.divide(100*sd_array,ModelParams03[0]) #Standard deviations as percentage of estimated parameter values, shows good fit

In [ ]:
###############################################################################
#                              Visualization 
###############################################################################

fig=plt.figure(4,figsize=(9,6))

LollipopPlot(fig,SolverTime,WhiteSignal,FitSolution3)

In [ ]:
inpData=np.linspace(0,10,20).reshape(-1,2)

In [ ]:
inpData.shape

In [ ]:
inpData.flatten()

In [ ]:
def func(X, a, b, c):
    x,y = X
    result0 = np.log(a) + b*np.log(x[0]) + c*np.log(y[0])
    result1 = np.log(a) + b*np.log(x[1]) + c*np.log(y[1])
    return np.concatenate([result0, result1])

# some artificially noisy data to fit
x0 = np.linspace(0.1,1.1,101)
y0 = np.linspace(1.,2., 101)

x1 = np.linspace(0.1,1.1,101)
y1 = np.linspace(1.,2., 101)

a, b, c = 10., 4., 6.

x = np.array([x0,x1])
y = np.array([y0,y1])

z = func((x,y), a, b, c)
# z[0] = z[0] * 1 + np.random.random(101)/100
# z[1] = z[1] * 1 + np.random.random(101)/100
z = z + np.random.random(2*101)/100
# initial guesses for a,b,c:
p0 = 8., 2., 7.
print(curve_fit(func, (x,y), z, p0))


In [ ]:
Ca0 = 1.0
k1 = k_1 = 3.0

tspan = np.linspace(0, 0.5,10)

def C(K):
    k1, k_1 = K
    def dCdt(Ca, t, K):
        k1,k_1 = K
        return -k1 * Ca + k_1 * (Ca0 - Ca)
    sol = odeint(dCdt, Ca0, tspan, tuple((k1, k_1)))
    return sol

plt.plot(tspan, C([k1, k_1]))
plt.xlim([tspan.min(), tspan.max()])
plt.xlabel('t')
plt.ylabel('C');

In [ ]:
np.asarray(C([k1, k_1])) 

In [ ]:
j = jacrev(C)

In [ ]:
np.asarray(j([k1,k_1])).shape

In [ ]:
import time
t0 = time.time()
dCdk = jacobian(C, 0)


k_sensitivity = dCdk(np.array([k1, k_1]))

k1_sensitivity = k_sensitivity[:, 0, 0]
k_1_sensitivity = k_sensitivity[:, 0, 1]

plt.plot(tspan, np.abs(k1_sensitivity), label='dC/dk1')
plt.plot(tspan, np.abs(k_1_sensitivity), label='dC/dk_1')
plt.legend(loc='best')
plt.xlabel('t')
plt.ylabel('sensitivity')
print(f'Elapsed time = {time.time() - t0:1.1f} seconds')

In [ ]:
k_sensitivity.shape

In [ ]:
import autograd.numpy as np
from autograd import jacobian

def my_add(K):
    a,b,c,d = K
    return a + b + c + 2*d

a0 = np.array([1.0, 2.0])
b0 = np.array([3.0, 4.0])
c0 = np.array([3.0, 4.0])
d0 = np.array([3.0, 4.0])
my_add_j = jacobian(my_add,argnum=(0))


In [ ]:
jacoutput = my_add_j(np.array([a0, b0,c0,d0]))

In [ ]:
jacoutput[0,:,:].shape

In [ ]:
import autograd.numpy as np
from autograd import jacobian

def my_add(a, b):
    return a + b

a0 = np.array([1.0, 2.0])
b0 = np.array([3.0, 4.0])
my_add_j = jacobian(my_add, argnum=(0, 1))
my_add_j(a0, b0)